<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

### Corpus

In [1]:
## SkLearn # Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### Custom implementation of TFIDF Vectorizer with out using Sklearn 

In [8]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [9]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm import tqdm
import os
import math 

In [10]:
from tqdm import tqdm 

# it accepts only list of sentances
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        idf_word=dict()
        #once=set()
        len_dataset=0
        for row in dataset:
          len_dataset+=1
          once=set()
          for word in row.split(" "):
            if word not in once:
              once.add(word)
              try:
                idf_word[word]+=1
              except :
                idf_word[word]=1
        vocab = {j:i for i,j in enumerate(unique_words)}
        idf=dict()
        for row in dataset:
          for word in row.split(" "):
            idf[word]=1+(math.log((1+len_dataset)/(1+idf_word[word])))
        return vocab,idf
    else:
        print("you need to pass list of sentance")

In [11]:
vocab,idf = fit(corpus)
print("The vocab is :", end ='')
print(vocab)

The vocab is :{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}


In [12]:
#feature names 
vocab.keys()

dict_keys(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'])

In [13]:
print('IDF of the given word : ',end='')
print(idf)

IDF of the given word : {'this': 1.0, 'is': 1.0, 'the': 1.0, 'first': 1.5108256237659907, 'document': 1.2231435513142097, 'second': 1.916290731874155, 'and': 1.916290731874155, 'third': 1.916290731874155, 'one': 1.916290731874155}


In [14]:
# https://stackoverflow.com/questions/9919604/efficiently-calculate-word-frequency-in-a-string
# https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.sparse.csr_matrix.html

def transform(dataset,vocab,idf):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    tf=(freq/len(row.split()))
                    #idf=1+math.log((1+len_dataset)/(1+idf_word[word]))
                    values.append(tf*idf[word])
        return  csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")

In [15]:
strings = ["the method of lagrange multipliers is the economists workhorse for solving optimization problems",
           "the technique is a centerpiece of economic theory but unfortunately its usually taught poorly"]
vocab,idf = fit(corpus)
a=transform(corpus, vocab,idf)

100%|██████████| 4/4 [00:00<00:00, 11155.06it/s]


In [16]:
#shape of the output 
a.shape

(4, 9)

In [17]:
#sparse data 
print(a)

  (0, 1)	0.24462871026284194
  (0, 2)	0.3021651247531982
  (0, 3)	0.2
  (0, 6)	0.2
  (0, 8)	0.2
  (1, 1)	0.40771451710473655
  (1, 3)	0.16666666666666666
  (1, 5)	0.3193817886456925
  (1, 6)	0.16666666666666666
  (1, 8)	0.16666666666666666
  (2, 0)	0.3193817886456925
  (2, 3)	0.16666666666666666
  (2, 4)	0.3193817886456925
  (2, 6)	0.16666666666666666
  (2, 7)	0.3193817886456925
  (2, 8)	0.16666666666666666
  (3, 1)	0.24462871026284194
  (3, 2)	0.3021651247531982
  (3, 3)	0.2
  (3, 6)	0.2
  (3, 8)	0.2


In [18]:
a.toarray()

array([[0.        , 0.24462871, 0.30216512, 0.2       , 0.        ,
        0.        , 0.2       , 0.        , 0.2       ],
       [0.        , 0.40771452, 0.        , 0.16666667, 0.        ,
        0.31938179, 0.16666667, 0.        , 0.16666667],
       [0.31938179, 0.        , 0.        , 0.16666667, 0.31938179,
        0.        , 0.16666667, 0.31938179, 0.16666667],
       [0.        , 0.24462871, 0.30216512, 0.2       , 0.        ,
        0.        , 0.2       , 0.        , 0.2       ]])

### TFIDF Vector

In [19]:
#normalized matrix 
normalize(a.toarray(),norm='l2') #TFIDF Vectors

array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

### Comparing Custom implemented TFIDF Vectorizer with SkLearn Implementation

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [3]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [4]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [5]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [6]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [7]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task you have to modify your fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li>This task is similar to your previous task, just that here your vocabulary is limited to only top 50 features names based on their idf values. Basically your output will have exactly 50 columns and the number of rows will depend on the number of documents you have in your corpus.</li>
    <br>
    <li>Here you will be give a pickle file, with file name <strong>cleaned_strings</strong>. You would have to load the corpus from this file and use it as input to your tfidf vectorizer.</li>
    <br>
    <li>Steps to approach this task:
    <ol>
        <li> You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer, just like in the previous task. Additionally, here you have to limit the number of features generated to 50 as described above.</li>
        <li> Now sort your vocab based in descending order of idf values and print out the words in the sorted voacb after you fit your data. Here you should be getting only 50 terms in your vocab. And make sure to print idf values for each term in your vocab. </li>
        <li> Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html </li>
        <li> Now check the output of a single document in your collection of documents,  you can convert the sparse matrix related only to that document into dense matrix and print it. And this dense matrix should contain 1 row and 50 columns. </li>
        </ol>
    </li>
    <br>
   </ul>

### Implement max features functionality

In [20]:


import pickle
with open('/content/drive/MyDrive/aaic/cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [22]:
from collections import OrderedDict

In [23]:
from tqdm import tqdm
# it accepts only list of sentances
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        idf_word=dict()
        #once=set()
        len_dataset=0
        for row in dataset:
          len_dataset+=1
          once=set()
          for word in row.split(" "):
            if word not in once:
              once.add(word)
              try:
                idf_word[word]+=1
              except :
                idf_word[word]=1
        vocab = {j:i for i,j in enumerate(unique_words)}
        idf=dict()
        for row in dataset:
          for word in row.split(" "):
            idf[word]=1+(math.log((1+len_dataset)/(1+idf_word[word])))  
        data=OrderedDict(sorted(idf.items(),key=lambda t:t[1],reverse=True))
        #getting top fifty vocabulary words 
        key=list(data.keys())[:50]
        #sorting values in ordered dictionary 
        for i in range(len(data.keys())):
          if i>49:
            try:
              data[key[i]]=0
            except:
              pass  
        vocab = {j:i for i,j in enumerate(key)}       
        return vocab,data
    else:
        print("you need to pass list of sentance")

In [24]:
vocab,idf = fit(corpus)
print("The vocab is :", end ='')
print(vocab)

The vocab is :{'aimless': 0, 'distressed': 1, 'drifting': 2, 'nearly': 3, 'attempting': 4, 'artiness': 5, 'existent': 6, 'gerardo': 7, 'emptiness': 8, 'effort': 9, 'messages': 10, 'buffet': 11, 'science': 12, 'teacher': 13, 'baby': 14, 'owls': 15, 'florida': 16, 'muppets': 17, 'person': 18, 'overdue': 19, 'screenplay': 20, 'post': 21, 'practically': 22, 'structure': 23, 'tightly': 24, 'constructed': 25, 'vitally': 26, 'occurs': 27, 'content': 28, 'fill': 29, 'dozen': 30, 'highest': 31, 'superlative': 32, 'require': 33, 'puzzle': 34, 'solving': 35, 'fit': 36, 'pulls': 37, 'punches': 38, 'graphics': 39, 'number': 40, 'th': 41, 'insane': 42, 'massive': 43, 'unlockable': 44, 'properly': 45, 'aye': 46, 'rocks': 47, 'doomed': 48, 'conception': 49}


In [25]:
#features
print("the features are  : ", end =' ')
print(list(vocab.keys()))

the features are  :  ['aimless', 'distressed', 'drifting', 'nearly', 'attempting', 'artiness', 'existent', 'gerardo', 'emptiness', 'effort', 'messages', 'buffet', 'science', 'teacher', 'baby', 'owls', 'florida', 'muppets', 'person', 'overdue', 'screenplay', 'post', 'practically', 'structure', 'tightly', 'constructed', 'vitally', 'occurs', 'content', 'fill', 'dozen', 'highest', 'superlative', 'require', 'puzzle', 'solving', 'fit', 'pulls', 'punches', 'graphics', 'number', 'th', 'insane', 'massive', 'unlockable', 'properly', 'aye', 'rocks', 'doomed', 'conception']


In [26]:
print('IDF values of the given word : ', end='')
print(idf)

IDF values of the given word : OrderedDict([('aimless', 6.922918004572872), ('distressed', 6.922918004572872), ('drifting', 6.922918004572872), ('nearly', 6.922918004572872), ('attempting', 6.922918004572872), ('artiness', 6.922918004572872), ('existent', 6.922918004572872), ('gerardo', 6.922918004572872), ('emptiness', 6.922918004572872), ('effort', 6.922918004572872), ('messages', 6.922918004572872), ('buffet', 6.922918004572872), ('science', 6.922918004572872), ('teacher', 6.922918004572872), ('baby', 6.922918004572872), ('owls', 6.922918004572872), ('florida', 6.922918004572872), ('muppets', 6.922918004572872), ('person', 6.922918004572872), ('overdue', 6.922918004572872), ('screenplay', 6.922918004572872), ('post', 6.922918004572872), ('practically', 6.922918004572872), ('structure', 6.922918004572872), ('tightly', 6.922918004572872), ('constructed', 6.922918004572872), ('vitally', 6.922918004572872), ('occurs', 6.922918004572872), ('content', 6.922918004572872), ('fill', 6.922918

In [27]:
# https://stackoverflow.com/questions/9919604/efficiently-calculate-word-frequency-in-a-string
# https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.sparse.csr_matrix.html

def transform(dataset,vocab,idf):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    try:
                      tf=(freq/len(row.split()))
                      #df=1+math.log((1+len_dataset)/(1+idf_word[word]))
                      values.append(tf*idf[word])
                    except:
                      pass  
        return  csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")

In [28]:
vocab,idf = fit(corpus)

In [29]:
a=transform(corpus, vocab,idf)

100%|██████████| 746/746 [00:00<00:00, 62632.88it/s]


In [30]:
#output shape
a.shape

(746, 50)

In [31]:
#sparse format of output
print(a)

  (0, 0)	0.865364750571609
  (0, 1)	0.865364750571609
  (0, 2)	0.865364750571609
  (1, 3)	0.7692131116192079
  (2, 4)	0.36436410550383536
  (2, 5)	0.36436410550383536
  (2, 6)	0.36436410550383536
  (4, 7)	0.6922918004572872
  (5, 8)	0.6922918004572872
  (7, 9)	0.7692131116192079
  (7, 10)	0.7692131116192079
  (9, 11)	1.153819667428812
  (9, 12)	1.153819667428812
  (9, 13)	1.153819667428812
  (10, 14)	2.307639334857624
  (10, 15)	2.307639334857624
  (11, 16)	0.865364750571609
  (12, 17)	1.730729501143218
  (15, 18)	0.865364750571609
  (16, 19)	0.5325321541979132
  (17, 20)	0.49449414318377655
  (17, 21)	0.49449414318377655
  (18, 22)	0.9889882863675531
  (19, 23)	0.015769744885131828
  (19, 24)	0.015769744885131828
  (19, 25)	0.015769744885131828
  (19, 26)	0.015769744885131828
  (19, 27)	0.015769744885131828
  (19, 28)	0.015769744885131828
  (19, 29)	0.015769744885131828
  (19, 30)	0.015769744885131828
  (19, 31)	0.015769744885131828
  (19, 32)	0.015769744885131828
  (19, 33)	0.0157697

In [32]:
#tfidf matrix of given data
a.toarray()

array([[0.86536475, 0.86536475, 0.86536475, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])